In [1]:
import pandas as pd
import numpy as np
import osmnx as ox
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Rectangle
import data_preprocessing as dp
import mdp

In [2]:
url_FT_train = 'INTERACTION/DR_USA_Roundabout_FT_train.csv'
df_train = pd.read_csv(url_FT_train)

# add a new label unique_id using case_id and track_id
df_train['unique_id'] = df_train['case_id'].astype(int) * 100+ df_train['track_id']

# change length, width and psi_rad from NaN to 0 for pedestrain/bicycle
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'length'] = 0
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'width'] = 0
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'psi_rad'] = 0

### important!!!
# agent_type pedestrain/bicycle = -1
# agent_type car = 1
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'agent_type'] = -1
df_train.loc[df_train['agent_type'] == 'car', 'agent_type'] = 1
df_train

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width,unique_id
0,1.0,1,1,100,-1,1044.509,1012.175,0.103,-1.005,0.000,0.00,0.00,101
1,1.0,2,1,100,-1,1043.840,1013.348,-0.101,-1.362,0.000,0.00,0.00,102
2,1.0,3,1,100,-1,1007.638,1022.038,0.687,0.787,0.000,0.00,0.00,103
3,1.0,4,1,100,1,1036.888,976.415,-1.063,1.980,2.064,5.20,1.86,104
4,1.0,5,1,100,1,1038.206,993.477,-2.343,5.161,1.997,4.68,1.90,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4243282,9811.0,8,40,4000,1,1022.107,985.137,7.470,1.627,0.215,4.19,1.87,981108
4243283,9811.0,9,40,4000,1,996.468,986.201,7.374,-0.178,-0.024,4.64,1.90,981109
4243284,9811.0,10,40,4000,1,1019.894,1009.985,-7.780,-0.610,-3.063,4.72,1.87,981110
4243285,9811.0,11,40,4000,1,1028.592,1022.465,-1.619,-1.729,-2.324,4.49,1.83,981111


### Total number of cases

In [3]:
unique_cases = df_train['case_id'].nunique()
unique_cases


9811

### Use small amout of cases for testing

In [4]:
number_of_cases = 411

#df_train_small = df_train[df_train['case_id'] <= number_of_cases]
df_train_small = df_train[(df_train['case_id'] > number_of_cases) & (df_train['case_id'] <= number_of_cases + 1) & (df_train['timestamp_ms'] >= 2300)& (df_train['timestamp_ms'] < 2700)]
df_train_small 

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width,unique_id
180409,412.0,2,23,2300,1,995.749,999.848,-8.187,-4.149,-2.673,5.85,1.98,41202
180410,412.0,3,23,2300,1,966.704,991.421,-9.688,-1.500,-2.988,4.94,1.94,41203
180411,412.0,4,23,2300,1,1014.650,1009.204,-7.175,-1.531,-2.931,4.82,1.95,41204
180412,412.0,5,23,2300,1,1037.557,1008.977,-5.565,1.122,2.943,4.57,1.95,41205
180413,412.0,2,24,2400,1,994.918,999.459,-8.311,-4.067,-2.687,5.85,1.98,41202
180414,412.0,3,24,2400,1,965.734,991.278,-9.694,-1.461,-2.992,4.94,1.94,41203
180415,412.0,4,24,2400,1,1013.943,1009.007,-7.166,-1.747,-2.902,4.82,1.95,41204
180416,412.0,5,24,2400,1,1037.003,1009.100,-5.653,1.186,2.935,4.57,1.95,41205
180417,412.0,2,25,2500,1,994.074,999.078,-8.432,-3.978,-2.701,5.85,1.98,41202
180418,412.0,4,25,2500,1,1013.239,1008.790,-7.152,-1.962,-2.874,4.82,1.95,41204


In [5]:
v_max = max(df_train_small['vx']**2 + df_train_small['vy']**2) ** 0.5

In [6]:
# load uniqueTracks data
# uniqueTracks is a dictionary strore each unique object
uniqueTracks = dp.read_uniqueTracks(df_train_small)
uniqueTracks

{41202: <data_preprocessing.uniqueTrack at 0x135915a90>,
 41203: <data_preprocessing.uniqueTrack at 0x135914770>,
 41204: <data_preprocessing.uniqueTrack at 0x135858cb0>,
 41205: <data_preprocessing.uniqueTrack at 0x135914290>}

In [7]:
# case 1: the ego is in the current time frame
ego_id = 1010
current_time = 100

# get other vehicles with same case and time
other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
print("other agents:", other_agents)
print("interactive agens:", interactive_agents)

other agents: []
interactive agens: []


In [8]:
total_agents = 0
non_car_agents = []
car_agents = []
collisions = []
for ego_id in uniqueTracks:
    total_agents += 1
    for current_time in range(100, 4001, 100):
        other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
        # print(ego_id)
        mdp_tuple = mdp.get_mdp_tuple(ego_id, interactive_agents, current_time, uniqueTracks, v_max)
        if not mdp_tuple: # if interactive_agents is empty or ego is not a car, then mdp_tuple is None
            #print(mdp_tuple)
            if ego_id not in non_car_agents:
                if uniqueTracks[ego_id].agent_type == -1:
                    non_car_agents.append(ego_id)
        else:
            if ego_id not in car_agents:
                if uniqueTracks[ego_id].agent_type == 1:
                    car_agents.append(ego_id)
            s, a, r, s_next, the_last_episode = mdp_tuple
            if r < 0:
                if ego_id not in collisions: 
                    collisions.append(ego_id)
                    #print(uniqueTracks[ego_id].agent_type)
                    #print(r)
                    #print(ego_id)
            if the_last_episode:
                break
print('Total agents: ', total_agents)
print('Total car agents: ', len(car_agents))
print('Total nor-car agents: ', len(non_car_agents))
print('Total collisions: ', collisions)

41202 [41204, 41203, 41205]
41202 [41204, 41203, 41205]
41202 [41204, 41205, 0]
41203 [41202, 41204, 41205]
41204 [41202, 41205, 41203]
41204 [41202, 41205, 41203]
41204 [41202, 41205, 0]
41205 [41204, 41202, 41203]
41205 [41204, 41202, 41203]
41205 [41204, 41202, 0]
Total agents:  4
Total car agents:  4
Total nor-car agents:  0
Total collisions:  []


/Users/kycheng/Desktop/5_Winter2024/1_EECS 545/project/github/EECS-545-final-project/reward.py:163: RuntimeWarning: invalid value encountered in arccos
  alpha = np.arccos((distance_next**2 + distance_closest**2 - distance_next_closest**2)/(2*distance_next*distance_closest))
/Users/kycheng/Desktop/5_Winter2024/1_EECS 545/project/github/EECS-545-final-project/reward.py:119: RuntimeWarning: invalid value encountered in arccos
  if rotated_rectangles_intersect(((ego_x, ego_y), 0.95 * ego_length, 0.95 * ego_width, np.arccos(ego_psi)),
/Users/kycheng/Desktop/5_Winter2024/1_EECS 545/project/github/EECS-545-final-project/reward.py:120: RuntimeWarning: invalid value encountered in arccos
  ((interactive_x, interactive_y), 0.95 * interactive_length, 0.95 * interactive_width, np.arccos(interactive_psi))):


In [9]:
#print('Total number of car agents: ', len(df_train_small['unique_id'][df_train['agent_type'] == 1].unique()))
#print('Total number of non-car agents: ', len(df_train_small['unique_id'][df_train['agent_type'] == -1].unique()))

### edge case: ego appears at the last time frame

In [10]:
#df_train_small[df_train_small['unique_id'] == 1214]

In [11]:
number_of_cases = 411

#df_train_small = df_train[df_train['case_id'] <= number_of_cases]
df_train_small2 = df_train[(df_train['case_id'] > number_of_cases) & (df_train['case_id'] <= number_of_cases + 1) & (df_train['timestamp_ms'] >= 2000)]
df_train_small2

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width,unique_id
180397,412.0,2,20,2000,1,998.169,1001.067,-7.823,-4.345,-2.635,5.85,1.98,41202
180398,412.0,3,20,2000,1,969.613,991.876,-9.691,-1.628,-2.975,4.94,1.94,41203
180399,412.0,4,20,2000,1,1016.778,1009.666,-7.162,-0.892,-3.018,4.82,1.95,41204
180400,412.0,5,20,2000,1,1039.157,1008.648,-5.235,0.922,2.967,4.57,1.95,41205
180401,412.0,2,21,2100,1,997.375,1000.653,-7.940,-4.289,-2.646,5.85,1.98,41202
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180474,412.0,6,39,3900,1,1037.281,1009.369,-6.201,1.762,2.865,4.59,1.93,41206
180475,412.0,2,40,4000,1,980.623,994.515,-9.001,-2.476,-2.873,5.85,1.98,41202
180476,412.0,4,40,4000,1,1002.894,1003.624,-6.994,-4.171,-2.604,4.82,1.95,41204
180477,412.0,5,40,4000,1,1027.180,1011.258,-6.613,1.154,2.969,4.57,1.95,41205


In [12]:
ego_id = 41202
current_time = 2200

# get other vehicles with same case and time
other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
print("other agents:", other_agents)
print("interactive agents:", interactive_agents)


other agents: []
interactive agents: []


In [13]:
mdp_tuple = mdp.get_mdp_tuple(ego_id, interactive_agents, current_time, uniqueTracks, v_max)
if not mdp_tuple: # if interactive_agents is empty or ego is not a car, then mdp_tuple is None
    print(mdp_tuple)
else:
    s, a, r, s_next, the_last_episode = mdp_tuple
    print('s: ', s)
    print('a: ', a)
    print('r: ', r)
    print('s_next: ', s_next)
    print('the_last_episode: ', the_last_episode)

None


In [14]:
#df_train_small[df_train_small['unique_id'] == 602]

In [15]:
ego_id = 159803
current_time = 100

# get other vehicles with same case and time
other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
print("other agents:", other_agents)
print("interactive agens:", interactive_agents)

other agents: []
interactive agens: []


In [16]:
# close case: 231201
# 

In [17]:
data = df_train_small[df_train_small['unique_id'] == 231201]
data2 = df_train_small[df_train_small['unique_id'] == 231202]
data3 = df_train_small[df_train_small['unique_id'] == 231203]
data4 = df_train_small[df_train_small['unique_id'] == 231204]
#data5 = df_train_small[df_train_small['unique_id'] == 606]

fig, ax = plt.subplots()
xdata, ydata = [], []
xdata2, ydata2 = [], []
xdata3, ydata3 = [], []
xdata4, ydata4 = [], []
#xdata5, ydata5 = [], []
ln, = plt.plot([], [], 'ro')  # 'ro' 表示紅色的圓點
ln2, = plt.plot([], [], 'bo')
ln3, = plt.plot([], [], 'go')
ln4, = plt.plot([], [], 'ko')
#ln5, = plt.plot([], [], 'rs')

def init():
    ax.set_xlim(950, 1060)  # 根據數據的 x 值範圍設定
    ax.set_ylim(950, 1020)  # 根據數據的 y 值範圍設定
    return ln, ln2, ln3, ln4#, ln5

def update(frame):
    xdata.append(data.iloc[frame]['x'])
    ydata.append(data.iloc[frame]['y'])
    ln.set_data(xdata, ydata)

    if frame < len(data2):
        xdata2.append(data2.iloc[frame]['x'])
        ydata2.append(data2.iloc[frame]['y'])
        ln2.set_data(xdata2, ydata2)
    else:
        ln2.set_data(xdata2, ydata2)
    #'''
    if frame < len(data3):
        xdata3.append(data3.iloc[frame]['x'])
        ydata3.append(data3.iloc[frame]['y'])
        ln3.set_data(xdata3, ydata3)
    else:
        ln3.set_data(xdata3, ydata3)

    if frame < len(data4):
        xdata4.append(data4.iloc[frame]['x'])
        ydata4.append(data4.iloc[frame]['y'])
        ln4.set_data(xdata4, ydata4)
    else:
        ln4.set_data(xdata4, ydata4)
    '''
    if frame < len(data5):
        xdata5.append(data5.iloc[frame]['x'])
        ydata5.append(data5.iloc[frame]['y'])
        ln5.set_data(xdata5, ydata5)
    else:
        ln5.set_data(xdata5, ydata5)
    '''
    
    return ln, ln2, ln3, ln4#, ln5

ani = animation.FuncAnimation(fig, update, frames=range(len(data)),
                    init_func=init, blit=False)

#plt.show()
# To display the animation in a Jupyter notebook you can use the following:
plt.close(fig)  # Avoid displaying the static plot
from IPython.display import HTML
HTML(ani.to_jshtml())  # Convert the animation to JSHTML and display it
